In [19]:
import requests

In [20]:
import geopandas as gpd
import pandas as pd

## Open State data

In [21]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Columm Data

In [22]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/subject/variables.json")
columns_obj = r.json()

In [23]:
table = "B12505"

url = "https://api.census.gov/data/2023/acs/acs5"
params = {
    "get": f"group({table})",
    "ucgid": "pseudo(0100000US$0400000)",
}
response = requests.get(url, params=params)

In [24]:
data = response.json()
columns = data[0]
rows = data[1:]
marriage_df = pd.DataFrame(rows, columns=columns)

In [25]:
carpool_cols_dict = {
    col_name: col_dict
    for col_name, col_dict in columns_obj["variables"].items()
    if col_dict["label"].split("!!")[-1] == "Carpooled" and col_name.startswith(table)
}

In [26]:
url = "https://api.census.gov/data/2023/acs/acs5/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [27]:
col_vars = {k: v["label"] for k, v in variables.items() if k.startswith(table)}
col_vars["ucgid"] = "AFFGEOID"

In [28]:
marriage_df = marriage_df.rename(columns=col_vars)
marriage_df = marriage_df[list(col_vars.values())]

In [30]:
marriage_df[["Estimate!!Total:"]] = marriage_df[["Estimate!!Total:"]].astype(int)
marriage_df[["Estimate!!Total:!!Male:!!Never married"]] = marriage_df[
    ["Estimate!!Total:!!Male:!!Never married"]
].astype(int)
marriage_df[["Estimate!!Total:!!Female:!!Never married"]] = marriage_df[
    ["Estimate!!Total:!!Female:!!Never married"]
].astype(int)

In [31]:
marriage_df["PERCENT"] = (
    (
        (
            marriage_df["Estimate!!Total:!!Male:!!Never married"]
            + marriage_df["Estimate!!Total:!!Female:!!Never married"]
        )
        / marriage_df["Estimate!!Total:"]
    )
    * 100
).round(decimals=1)

## Merge Data

In [32]:
marriage_gdf = state_gdf.merge(marriage_df, on="AFFGEOID", how="inner")

In [33]:
marriage_gdf = marriage_gdf.to_crs(9311)
marriage_gdf.to_file("data/Never_Married_Per_State.gpkg")